We use some nice functions from Fabien Daniel to load the data and get a good read on the genres, so we can summarize these. All credits from especially the first part of this notebook are due: https://www.kaggle.com/fabiendaniel/categorizing-actors-hands-on-plotly

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import json

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import Imputer

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
movies = pd.read_csv('../input/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb_5000_credits.csv')

In [ ]:
del credits['title']
df = pd.concat([movies, credits], axis=1)

In [ ]:
df.head()

We use some functions from Fabien Daniel to load the json columns correctly and so we can perform a nice split on the genre column to extract different genres.

In [ ]:
def load_tmdb_movies(path):
    df = pd.read_csv(path)
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.date())
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

def load_tmdb_credits(path):
    df = pd.read_csv(path)
    json_columns = ['cast', 'crew']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])

In [ ]:
credits = load_tmdb_credits("../input/tmdb_5000_credits.csv")
movies = load_tmdb_movies("../input/tmdb_5000_movies.csv")

In [ ]:
del credits['title']
df = pd.concat([movies, credits], axis=1)

In [ ]:
df['genres'] = df['genres'].apply(pipe_flatten_names)

In [ ]:
liste_genres = set()
for s in df['genres'].str.split('|'):
    liste_genres = set().union(s, liste_genres)
liste_genres = list(liste_genres)
liste_genres.remove('')

Now we can take a good look at the genres. We resize our dataframe, so we keep only our most important columns:

In [ ]:
df_reduced = df[['title','vote_average','release_date','runtime','budget','revenue']].reset_index(drop=True)

In [ ]:
df_reduced.head()

Now we attach all the different genres to our reduced data frame and add a 1 or a 0 if the given movie is of a given genre.

In [ ]:
for genre in liste_genres:
    df_reduced[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)
df_reduced[:5]

Fabien Daniel shows us a nice way to create a pie chart to get an overview of the representation of every genre in the data frame.

In [ ]:
plt.rc('font', weight='bold')
f, ax = plt.subplots(figsize=(5,5))
genre_count = []
for genre in liste_genres:
    genre_count.append([genre, df_reduced[genre].values.sum()])
genre_count.sort(key = lambda x:x[1], reverse = True)
labels, sizes = zip(*genre_count)
labels_selected = [n if v > sum(sizes) * 0.01 else '' for n, v in genre_count]
ax.pie(sizes, labels=labels_selected,
      autopct = lambda x:'{:2.0f}%'.format(x) if x>1 else '',
      shadow = False, startangle=0)
ax.axis('equal')
plt.tight_layout()

Now we want to calculate all the averages per genre. We create a data frame consisting of only one column containing every genre and add the mean of the vote_average, the mean budget and the mean revenue.

In [ ]:
mean_per_genre = pd.DataFrame(liste_genres)

In [ ]:
newArray = []*len(liste_genres)
for genre in liste_genres:
    newArray.append(df_reduced.groupby(genre, as_index=True)['vote_average'].mean())
newArray2 = []*len(liste_genres)
for i in range(len(liste_genres)):
    newArray2.append(newArray[i][1])

mean_per_genre['mean_votes_average']=newArray2

We apply the same strategy for both budget and revenue:

In [ ]:
newArray = []*len(liste_genres)
for genre in liste_genres:
    newArray.append(df_reduced.groupby(genre, as_index=True)['budget'].mean())
newArray2 = []*len(liste_genres)
for i in range(len(liste_genres)):
    newArray2.append(newArray[i][1])

mean_per_genre['mean_budget']=newArray2

In [ ]:
newArray = []*len(liste_genres)
for genre in liste_genres:
    newArray.append(df_reduced.groupby(genre, as_index=True)['revenue'].mean())
newArray2 = []*len(liste_genres)
for i in range(len(liste_genres)):
    newArray2.append(newArray[i][1])

mean_per_genre['mean_revenue']=newArray2

Let's calculate the 'mean profit' per genre:

In [ ]:
mean_per_genre['profit'] = mean_per_genre['mean_revenue']-mean_per_genre['mean_budget']

In [ ]:
mean_per_genre

In [ ]:
mean_per_genre['mean_votes_average'].plot.barh()

We want to look at averages per genre per year, so let's first make a selection of every year of release

In [ ]:
from datetime import datetime
#list(map(datetime.year, df_reduced["release_date"]))
t = df_reduced['release_date']
t = pd.to_datetime(t)
t = t.dt.year
df_reduced['release_year'] = t

Now let's create a seperate data frame for every genre and calculate all the averages for every year a movie in that genre was released.

In [ ]:
df_list = []*len(liste_genres)
for genre in liste_genres:
    df_list.append(df_reduced.groupby([genre,'release_year']).mean().reset_index())

The data frames created above are grouped by the 0 and 1 values in a genre column. Let's keep only the rows where the genre equals 1. So where the data frame is of the specific genre.

In [ ]:
df_per_genre = []*len(liste_genres)
for i in range(len(df_list)):
    df_per_genre.append(df_list[i][df_list[i].ix[:,0] == 1])

Now we can create data frames which give the average budget, revenue and vote_average per genre per year. We start looking at the year 2000.

In [ ]:
columns = range(1988,2018)
budget_genre = pd.DataFrame( columns = columns)
budget_genre

In [ ]:
for genre in liste_genres:
    temp=(df_per_genre[liste_genres.index(genre)].pivot_table(index = genre, columns = 'release_year', values = 'budget', aggfunc = np.mean))
    temp = temp[temp.columns[-30:]].loc[1]
    budget_genre.loc[liste_genres.index(genre)]=temp
budget_genre['genre']=liste_genres

In [ ]:
budget_genre.index = budget_genre['genre']
budget_genre

In [ ]:
columns = range(1988,2018)
revenue_genre = pd.DataFrame( columns = columns)
revenue_genre

In [ ]:
for genre in liste_genres:
    temp=(df_per_genre[liste_genres.index(genre)].pivot_table(index = genre, columns = 'release_year', values = 'revenue', aggfunc = np.mean))
    temp = temp[temp.columns[-30:]].loc[1]
    revenue_genre.loc[liste_genres.index(genre)]=temp
revenue_genre['genre']=liste_genres

In [ ]:
revenue_genre.index = revenue_genre['genre']
revenue_genre

In [ ]:
columns = range(1988,2018)
vote_avg_genre = pd.DataFrame( columns = columns)
vote_avg_genre

In [ ]:
for genre in liste_genres:
    temp=(df_per_genre[liste_genres.index(genre)].pivot_table(index = genre, columns = 'release_year', values = 'vote_average', aggfunc = np.mean))
    temp = temp[temp.columns[-30:]].loc[1]
    vote_avg_genre.loc[liste_genres.index(genre)]=temp
vote_avg_genre['genre']=liste_genres

In [ ]:
vote_avg_genre.index = vote_avg_genre['genre']
vote_avg_genre

We can create more insight in these tables by making heatmaps. Budget has a few outliers in western which we may remove to get a better overview of the distribution.

In [ ]:
len(budget_genre)

In [ ]:
#f, [axA, axB] = plt.subplots(figsize = (9, 9), nrows = 2)
fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(budget_genre.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)
#axA.set_ylabel('genres')
#df.ix[:,0:2]

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(revenue_genre.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(vote_avg_genre.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)

In [ ]:
temp = budget_genre
temp

In [ ]:

temp[2013]=temp[2013].replace(2.550000e+08, 0)

In [ ]:
temp

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(temp.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)

Now it is way more obvious that Adventure, Fantasy, Science fiction and Animation movies have got on average the highest budget. The increase in budget through the years is alsno nice to see.

In [ ]:
df_genre = pd.DataFrame(columns = ['genre', 'cgenres', 'budget', 'gross', 'year'])

In [ ]:
from datetime import datetime
#list(map(datetime.year, df_reduced["release_date"]))
t = df['release_date']
t = pd.to_datetime(t)
t = t.dt.year
df_genre['release_year'] = t

In [ ]:
colnames = ['budget', 'genres', 'revenue']
df_clean = df[colnames]

In [ ]:
df_clean['release_year'] = t

In [ ]:
df_clean = df_clean.dropna()
df_genre = df_genre.dropna()

In [ ]:
df_clean.head()

In [ ]:
def genreRemap(row):
    global df_genre
    d = {}
    genres = np.array(row['genres'].split('|'))
    n = genres.size
    d['budget'] = [row['budget']]*n
    d['revenue'] = [row['revenue']]*n
    d['year'] = [row['release_year']]*n
    d['genre'], d['cgenres'] = [], []
    for genre in genres:
        d['genre'].append(genre)
        d['cgenres'].append(genres[genres != genre])
    df_genre = df_genre.append(pd.DataFrame(d), ignore_index = True)

df_clean.apply(genreRemap, axis = 1)
df_genre['year'] = df_genre['year'].astype(np.int16)
df_genre = df_genre[['genre', 'budget', 'gross', 'year', 'cgenres']]

In [ ]:
####################
# make connections #
####################
d_genre = {}
def connect(row):
    global d_genre
    genre = row['genre']
    cgenres = row['cgenres']
    if genre not in d_genre:
        d_cgenres = dict(zip(cgenres, [1]*len(cgenres)))
        d_genre[genre] = d_cgenres
    else:
        for cgenre in cgenres:
            if cgenre not in d_genre[genre]:
                d_genre[genre][cgenre] = 1
            else:
                d_genre[genre][cgenre] += 1
                
df_genre.apply(connect, axis = 1)
l_genre = list(d_genre.keys())
l_genre.sort()
###########################
# find largest connection #
###########################
cmax = 0
for key in d_genre:
    for e in d_genre[key]:
        if d_genre[key][e] > cmax:
            cmax = d_genre[key][e]
#########################
# visualize connections #
#########################
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib import cm
color = cm.get_cmap('rainbow')
f, ax = plt.subplots(figsize = (7, 9))

codes = [Path.MOVETO, Path.CURVE4, Path.CURVE4, Path.CURVE4]

X, Y = 1, 1
wmin, wmax = 1, 32
amin, amax = 0.1, 0.25
getPy = lambda x: Y*(1 - x/len(l_genre))
for i, genre in enumerate(l_genre):
    yo = getPy(i)
    ax.text(0, yo, genre, ha = 'right')
    ax.text(X, yo, genre, ha = 'left')
    for cgenre in d_genre[genre]:
        yi = getPy(l_genre.index(cgenre))
        verts = [(0.0, yo), (X/4, yo), (2*X/4, yi), (X, yi)]
        path = Path(verts, codes)
        r, g, b, a = color(i/len(l_genre))
        width = wmin + wmax*d_genre[genre][cgenre]/cmax
        alpha = amin + amax*(1 - d_genre[genre][cgenre]/cmax)
        patch = patches.PathPatch(path, facecolor = 'none', edgecolor = (r, g, b), lw = width, alpha = alpha)
        ax.add_patch(patch)

ax.grid(False)
ax.set_xlim(0.0, X)
ax.set_ylim(0.0, Y + 1/len(l_genre))
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.show()